In [1]:
from dateutil.parser import *
import pandas as pd

from clan_tools.data_adapters.YQLAdapter import YQLAdapter
yql_adapter = YQLAdapter()

from clan_tools.data_adapters.ClickHouseYTAdapter import ClickHouseYTAdapter
ch_adapter = ClickHouseYTAdapter()

from clan_tools.data_adapters.YTAdapter import YTAdapter
yt_adapter = YTAdapter()

In [2]:
query = """SELECT 
            cube.billing_account_id AS billing_account_id,
            SUM(real_consumption) AS real_consumption_mdb,
            SUM(trial_consumption) AS trial_consumption_mdb
        FROM
        (SELECT 
            distinct billing_account_id
            FROM `//home/cloud_analytics/cubes/acquisition_cube/cube`
            WHERE 
                event = 'cloud_created' --'ba_created'
                AND event_time>='2020-09-01'
                AND event_time<'2021-03-01'
                AND segment_actual IN ('Mass', 'Medium')) AS ba
        INNER JOIN `//home/cloud_analytics/cubes/acquisition_cube/cube` AS cube ON cube.billing_account_id = ba.billing_account_id
        WHERE
            service_name = 'mdb'
            AND event = 'day_use'
            AND cube.billing_account_id NOT IN (SELECT ba_id FROM `//home/cloud_analytics/kulaga/oppty_cube`)
        GROUP BY 
            billing_account_id
"""

df_mdb = ch_adapter.execute_query(query, to_pandas=True)


query = """SELECT 
            cube.billing_account_id AS billing_account_id,
            SUM(real_consumption)+SUM(trial_consumption) AS all_consumption_compute
        FROM
        (SELECT 
            distinct billing_account_id
            FROM `//home/cloud_analytics/cubes/acquisition_cube/cube`
            WHERE 
                event = 'cloud_created' --'ba_created'
                AND event_time>='2020-09-01'
                AND event_time<'2021-03-01'
                AND segment_actual IN ('Mass', 'Medium')) AS ba
        INNER JOIN `//home/cloud_analytics/cubes/acquisition_cube/cube` AS cube ON cube.billing_account_id = ba.billing_account_id
        WHERE
            service_name = 'compute'
            AND event = 'day_use'
            AND cube.billing_account_id NOT IN (SELECT ba_id FROM `//home/cloud_analytics/kulaga/oppty_cube`)
        GROUP BY 
            billing_account_id
"""

df_compute = ch_adapter.execute_query(query, to_pandas=True)


In [3]:
df = pd.merge(df_compute, df_mdb, how='outer', on='billing_account_id')
df.fillna(0, inplace=True)

df.loc[(df.all_consumption_compute>0) &
  (df.real_consumption_mdb==0 )&
  (df.trial_consumption_mdb==0 ), 'list1'] = 1

df.loc[(df.real_consumption_mdb==0 )&
  (df.trial_consumption_mdb>0 ), 'list2'] = 1

df.fillna(0, inplace=True)

In [13]:
df.list1 = df.list1.astype('int')
df.list2 = df.list2.astype('int')

df_schema = [
            {"name": "billing_account_id", "type": "string"},
            {"name": "all_consumption_compute", "type": "double"},
            {"name": "real_consumption_mdb", "type": "double"},
            {"name": "trial_consumption_mdb", "type": "double"},
            {"name": "list1", "type": "int64"},
            {"name": "list2", "type": "int64"},
]
    
yt_adapter.save_result('//home/cloud_analytics/elena-nenova/CLOUDANA-1432/list', 
                        df=df,
                        schema=df_schema,
                        append=False
                        )